In [44]:
import os
import numpy as np
import pandas as pd

def filepaths(sesspath, region, signal):
    region = {'pfc': 'imec0', 'hpc': 'imec1'}[region.lower()]
    signal = {'ap': 'ap', 'lfp': 'lf'}[signal.lower()]
    basedir = sesspath + f'_{region}'
    basename = sesspath.split('/')[-1] + f'_t0.{region}.{signal}'
    datapath = os.path.join(basedir, basename + '.bin')
    metapath = os.path.join(basedir, basename + '.meta')
    return datapath, metapath

def metadata(metapath):
    with open(metapath, 'r') as f:
        md = {}
        for line in f.read().splitlines():
            key, val = line.split('=')
            if key.startswith('~'):
                key = key[1:]
                print(key)
            md[key] = val
    return md

def reconstruct_hpc_lfp_metadata(pfc_lfp, hpc_ap, sha1, sizebytes):
    new = hpc_ap.copy()
    new['snsSaveChanSubset'] = pfc_lfp['snsSaveChanSubset']
    new['snsApLfSy'] = pfc_lfp['snsApLfSy']
    new['imSampRate'] = pfc_lfp['imSampRate']
    new['snsChanMap'] = pfc_lfp['snsChanMap']
    new['fileName'] = hpc_ap['fileName'].replace('.ap', '.lf')
    new['firstSample'] = pfc_lfp['firstSample']
    new['fileSHA1'] = sha1.upper()
    new['fileSizeBytes'] = str(sizebytes)
    new['fileTimeSecs'] = hpc_ap['fileTimeSecs']
    del new['snsShankMap']
    return new

root = '/Users/lukearend/phd/kfx/'
prefix = root + 'data/mnt/Ketamine/'
sessions = pd.read_csv(root + 'ref/sessions.csv')

In [45]:
sess = sessions.iloc[6]
_, metapath = filepaths(sess.path, 'pfc', 'lfp')
pfc_lfp = metadata(prefix + metapath)
_, metapath = filepaths(sess.path, 'hpc', 'ap')
hpc_ap = metadata(prefix + metapath)

imroTbl
snsChanMap
imroTbl
snsChanMap
snsShankMap


In [46]:
new = reconstruct_hpc_lfp_metadata(
    pfc_lfp, hpc_ap,
    sha1='0a9f9c4585379a4866ae342e7f3c8f5085998e73',
    sizebytes=13964344240
)

In [47]:
lines = new.copy()
lines['~imroTbl'] = lines.pop('imroTbl')
lines['~snsChanMap'] = lines.pop('snsChanMap')
lines = [f'{k}={v}' for k, v in lines.items()]
txt = '\n'.join(lines)

In [48]:
_, outpath = filepaths(sess.path, 'hpc', 'lfp')
outname = outpath.split('/')[-1]

In [50]:
with open(root + 'ref/2022-08-08-04-05-00_M017_SAL_mPFC_HPC_0_0_0mpk_g0_t0.imec1.lf.meta', 'w') as f:
    f.write(txt)